# Get daily CA reservoir levels data 1988-today

### Import Python tools and Jupyter configuration

In [21]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [22]:
import pandas as pd
import altair as alt
import altair_latimes as lat
import datetime as dt

In [23]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

In [24]:
today = dt.datetime.today().strftime("%Y-%m-%d")

### Major reservoirs

In [25]:
reservoirs = pd.DataFrame(
    [
        {"name": "San Luis", "abbr": "SNL"},
        {"name": "Melones", "abbr": "NML"},
        {"name": "Trinity", "abbr": "CLE"},
        {"name": "Shasta", "abbr": "SHA"},
        {"name": "Oroville", "abbr": "ORO"},
        {"name": "Folson", "abbr": "FOL"},
        {"name": "Don Pedro", "abbr": "DNP"},
        {"name": "McClure", "abbr": "EXC"},
        {"name": "Pine Flat", "abbr": "PNF"},
        {"name": "Castaic", "abbr": "CAS"},
        {"name": "Perris", "abbr": "PRR"},
        {"name": "Millerton", "abbr": "MIL"},
    ]
)

### Get three decades of daily acre-feet storage levels data from each reservoir

In [26]:
# https://cdec.water.ca.gov/dynamicapp/req/JSONDataServlet?Stations=ORO&SensorNums=15&dur_code=D&Start=2021-05-25&End=2021-06-25

In [28]:
levels = []

for r in reservoirs["abbr"]:
    levels.append(
        pd.read_json(
            "http://cdec.water.ca.gov/dynamicapp/req/JSONDataServlet?Stations="
            + r
            + "&SensorNums=15&dur_code=D&Start=1991-01-01&End="
            + today
        )
    )

### Combine each reservoir table into one large dataframe

In [56]:
src = pd.concat(levels).reset_index(drop=True)

In [57]:
src.head()

,stationId,durCode,SENSOR_NUM,sensorType,date,obsDate,value,dataFlag,units
0,SNL,D,15,STORAGE,1991-01-01,1991-1-1 00:00,491232,,AF
1,SNL,D,15,STORAGE,1991-01-02,1991-1-2 00:00,495043,,AF
2,SNL,D,15,STORAGE,1991-01-03,1991-1-3 00:00,502621,,AF
3,SNL,D,15,STORAGE,1991-01-04,1991-1-4 00:00,512765,,AF
4,SNL,D,15,STORAGE,1991-01-05,1991-1-5 00:00,520760,,AF


### Clean up

In [58]:
src.columns = src.columns.str.lower()

In [59]:
src.drop(
    ["durcode", "sensor_num", "sensortype", "obsdate", "dataflag", "units"],
    axis=1,
    inplace=True,
)

In [60]:
src.rename(columns={"value": "storage_acrefeet"}, inplace=True)

### Dates

In [61]:
src["date"] = pd.to_datetime(src["date"], format="%Y-%m-%d")
src["date"] = pd.to_datetime(src["date"].dt.strftime("%Y-%m-%d"))
src["year"] = src["date"].dt.year
src["quarter"] = src["date"].dt.quarter
src["day"] = src["date"].dt.day
src["month"] = src["date"].dt.month
src["weekday"] = src["date"].dt.weekday
src["monthname"] = src["date"].dt.month_name()
src["monthday"] = src["date"].dt.strftime("%m-%d")

### Latest value is often -9999

In [62]:
src = src[src["date"] != src["date"].max()]

In [63]:
df = src.copy()

In [64]:
len(df)

127556

---

In [65]:
df.head()

,stationid,date,storage_acrefeet,year,quarter,day,month,weekday,monthname,monthday
0,SNL,1991-01-01,491232,1991,1,1,1,1,January,01-01
1,SNL,1991-01-02,495043,1991,1,2,1,2,January,01-02
2,SNL,1991-01-03,502621,1991,1,3,1,3,January,01-03
3,SNL,1991-01-04,512765,1991,1,4,1,4,January,01-04
4,SNL,1991-01-05,520760,1991,1,5,1,5,January,01-05


In [66]:
df.tail()

,stationid,date,storage_acrefeet,year,quarter,day,month,weekday,monthname,monthday
127562,MIL,2021-06-20,261121,2021,2,20,6,6,June,06-20
127563,MIL,2021-06-21,260330,2021,2,21,6,0,June,06-21
127564,MIL,2021-06-22,259342,2021,2,22,6,1,June,06-22
127565,MIL,2021-06-23,258360,2021,2,23,6,2,June,06-23
127566,MIL,2021-06-24,256693,2021,2,24,6,3,June,06-24


In [68]:
df["date_str"] = df["date"].astype(str)
df["monthday"] = df["monthday"].astype(str)

In [69]:
df.to_json(
    "data/processed/levels/historical_levels_" + today + ".json",
    indent=2,
    orient="records",
)

In [73]:
df[df["year"] == 2021].to_json(
    "data/processed/levels/historical_levels_2021.json",
    indent=2,
    orient="records",
)